In [1]:
import datetime
from datetime import date
import requests
import pprint
from bs4 import BeautifulSoup
import pandas as pd
import re
import pickle
from urllib.request import urlopen
import glob
# import altair as alt
import time
import itertools
import numpy as np
import operator

pd.options.display.max_columns = 100



/Users/srav88/anaconda3/lib/python3.7/site-packages/pandas/compat/_optional.py:138: UserWarning: Pandas requires version '2.7.0' or newer of 'numexpr' (version '2.6.9' currently installed).
  warnings.warn(msg, UserWarning)


In [19]:
# Load Datasets

df_1 = pd.read_csv('player_profile_df_1.csv')
df_2 = pd.read_csv('player_profile_df_2.csv')
df_3 = pd.read_csv('player_profile_df_3.csv')

profile_df = pd.concat([df_1, df_2])
# profile_df = pd.concat([profile_df, df_3])

player_injuries_df = pd.read_csv('player_injuries_df.csv')
player_weights_df = pd.read_csv('player_weights.csv')


len(df_2.columns)

41

In [12]:
# len(df_1.columns)

41

In [8]:
# df_3.columns

Index(['Place of birth:', 'Citizenship:', 'Height:', 'Position:',
       'Contract expires:', 'Former International:', 'Caps/Goals:',
       'League level:', 'Joined:', 'Contract until:', 'name', 'TMId',
       'Date of birth', 'Last club:', 'Most games for:', 'Retired since:',
       'club:', 'country:', 'Agent:', 'Without Club since:', 'Unknown since:',
       'National player:', 'Current international:', 'Date of death:',
       'Career break since:', 'Company:'],
      dtype='object')

In [14]:
len(profile_df)

8000

In [15]:
profile_df.head()

,Place of birth:,Citizenship:,Height:,Position:,Contract expires:,Former International:,Caps/Goals:,League level:,Joined:,Contract until:,name,TMId,Date of birth,Last club:,Most games for:,Retired since:,club:,Name in home country:,Date of birth:,Age:,Foot:,Player agent:,Current club:,Outfitter:,Social-Media:,Agent:,Without Club since:,Full name:,Contract option:,National player:,Current international:,country:,Career break since:,On loan from:,Contract there expires:,Unknown since:,Date of death:,2nd club:,3nd club:,Company:,Ban since:
0,Petr Čech,39,Plzeň,"1,96 m",Retired,Czechia,124/0,NaN,SPORT INVEST,NaN,Petr Cech,5658,NaN,Arsenal,Chelsea,"Jul 1, 2019",Advisor,Chelsea,Advisor,"May 20, 1982",Czech Republic,Goalkeeper,left,"Jul 1, 2019",-,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Scout,36,Bishop Auckland,"1,93 m",Retired,England U19,8/0,NaN,right,NaN,Ross Turnbull,29159,NaN,Leeds,NaN,"Jul 1, 2017",Scout,NaN,Chelsea,"Jan 4, 1985",England,NaN,Goalkeeper,NaN,"Jul 1, 2017",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Henrique Hilário Meireles Alves Sampaio,46,São Pedro da Cova,"1,89 m",Retired,Portugal,1/0,NaN,Gestifute,NaN,Henrique Hilário,13886,NaN,Chelsea,NaN,"Jul 1, 2014",Goalkeeping Coach,Chelsea,Goalkeeping Coach,"Oct 21, 1975",Portugal,Goalkeeper,right,NaN,NaN,Gestifute,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,John George Terry,40,London,"1,87 m",Retired,England,78/6,NaN,Elite Management Agency,NaN,John Terry,3160,NaN,Aston Villa,Chelsea,"Oct 7, 2018",Assistant Manager,Aston Villa,Assistant Manager,"Dec 7, 1980",England,Defender - Centre-Back,right,NaN,"Oct 7, 2018",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Sremska Mitrovica,Serbia,"1,85 m",Defender - Centre-Back,-,Serbia,105/13,NaN,"Jul 1, 2021",NaN,Branislav Ivanovic,36827,NaN,West Brom,Chelsea,NaN,NaN,Бранислав Ивановић,"Feb 22, 1984",37,right,Star Management Signings Ltd.,Without Club,NaN,NaN,NaN,"Jul 1, 2021",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
# Merge Profile Information with Injuries

profile_injuries_df = pd.merge(left=player_injuries_df, right=profile_df, left_on=['TMId'], right_on=['TMId'], how='left')

In [17]:
len(player_injuries_df), len(profile_injuries_df)

(55216, 55216)

In [45]:
player_weights_df.sort_values(by=['Name', 'year'], inplace=True)


messi = player_weights_df[player_weights_df['Name']=='L. Messi']

messi.loc[messi['year'].str[-2:].astype(int) - 1 < 10, 'season'] = '0' + (messi['year'].str[-2:].astype(int) - 1).astype(str) + '/' + (messi['year'].str[-2:])
messi.loc[messi['year'].str[-2:].astype(int) - 1 >= 10, 'season'] = (messi['year'].str[-2:].astype(int) - 1).astype(str) + '/' + (messi['year'].str[-2:])
messi['Surname'] = messi['Name'].str.split().str[-1]


messi.bfill(axis=0)

/Users/srav88/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:1681: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = empty_value
/Users/srav88/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:1773: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)
/Users/srav88/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using 

,Name,Age,Overall Rating,Potential,Team,Contract expiry,Height,Weight,Strong foot,Value,Full Name,date,year,season,Surname
36102,L. Messi,20.0,86.0,93.0,FC Barcelona,2014,169cm,67kg,Left,€0,Lionel Messi,"Feb 22, 2007",FIFA 07,06/07,Messi
33656,L. Messi,20.0,86.0,93.0,FC Barcelona,2014,169cm,67kg,Left,€0,Lionel Messi,"Feb 22, 2008",FIFA 08,07/08,Messi
31203,L. Messi,21.0,90.0,95.0,FC Barcelona,2014,169cm,67kg,Left,€0,Lionel Messi,"Feb 22, 2009",FIFA 09,08/09,Messi
28800,L. Messi,22.0,90.0,94.0,FC Barcelona,2016,169cm,67kg,Left,€0,Lionel Messi,"Feb 22, 2010",FIFA 10,09/10,Messi
26400,L. Messi,23.0,90.0,94.0,FC Barcelona,2000 ~ 2016,169cm,67kg,Left,€0,Lionel Messi,"Feb 22, 2011",FIFA 11,10/11,Messi
24000,L. Messi,24.0,94.0,96.0,FC Barcelona,2004 ~ 2016,169cm,67kg,Left,€92.5M,Lionel Messi,"Feb 22, 2012",FIFA 12,11/12,Messi
21600,L. Messi,25.0,94.0,97.0,FC Barcelona,2004 ~ 2016,169cm,67kg,Left,€107.5M,Lionel Messi,"Sep 20, 2013",FIFA 13,12/13,Messi
19200,L. Messi,26.0,94.0,97.0,FC Barcelona,2004 ~ 2016,169cm,67kg,Left,€78M,Lionel Messi,"Sep 19, 2014",FIFA 14,13/14,Messi
16800,L. Messi,27.0,93.0,93.0,FC Barcelona,2004 ~ 2018,170cm,72kg,Left,€100.5M,Lionel Messi,"Sep 10, 2015",FIFA 15,14/15,Messi
14400,L. Messi,28.0,94.0,94.0,FC Barcelona,2004 ~ 2018,170cm,72kg,Left,€101M,Lionel Messi,"Sep 22, 2016",FIFA 16,15/16,Messi


In [40]:
profile_injuries_df[profile_injuries_df['name_x'] == 'Lionel Messi']

,Season,Injury,from,until,Days,Games missed,TMId,name_x,Place of birth:,Citizenship:,Height:,Position:,Contract expires:,Former International:,Caps/Goals:,League level:,Joined:,Contract until:,name_y,Date of birth,Last club:,Most games for:,Retired since:,club:,Name in home country:,Date of birth:,Age:,Foot:,Player agent:,Current club:,Outfitter:,Social-Media:,Agent:,Without Club since:,Full name:,Contract option:,National player:,Current international:,country:,Career break since:,On loan from:,Contract there expires:,Unknown since:,Date of death:,2nd club:,3nd club:,Company:,Ban since:
32691,21/22,Knee Problems,"Oct 29, 2021","Nov 11, 2021",13 days,3,28003,Lionel Messi,Rosario,Argentina Spain,"1,69 m",attack - Right Winger,"Jun 30, 2023",NaN,158/80,First Tier,"Aug 10, 2021","Jun 30, 2023",Lionel Messi,NaN,NaN,NaN,NaN,NaN,Lionel Andrés Messi Cuccitini,"Jun 24, 1987",34,left,Relatives,Paris Saint-Germain,adidas,NaN,NaN,NaN,NaN,Option for a further year,NaN,Argentina,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
32692,21/22,Bone Bruise,"Sep 20, 2021","Sep 27, 2021",7 days,2,28003,Lionel Messi,Rosario,Argentina Spain,"1,69 m",attack - Right Winger,"Jun 30, 2023",NaN,158/80,First Tier,"Aug 10, 2021","Jun 30, 2023",Lionel Messi,NaN,NaN,NaN,NaN,NaN,Lionel Andrés Messi Cuccitini,"Jun 24, 1987",34,left,Relatives,Paris Saint-Germain,adidas,NaN,NaN,NaN,NaN,Option for a further year,NaN,Argentina,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
32693,21/22,Fitness,"Aug 1, 2021","Aug 26, 2021",25 days,4,28003,Lionel Messi,Rosario,Argentina Spain,"1,69 m",attack - Right Winger,"Jun 30, 2023",NaN,158/80,First Tier,"Aug 10, 2021","Jun 30, 2023",Lionel Messi,NaN,NaN,NaN,NaN,NaN,Lionel Andrés Messi Cuccitini,"Jun 24, 1987",34,left,Relatives,Paris Saint-Germain,adidas,NaN,NaN,NaN,NaN,Option for a further year,NaN,Argentina,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
32694,20/21,Hamstring Injury,"Jan 12, 2021","Jan 15, 2021",3 days,1,28003,Lionel Messi,Rosario,Argentina Spain,"1,69 m",attack - Right Winger,"Jun 30, 2023",NaN,158/80,First Tier,"Aug 10, 2021","Jun 30, 2023",Lionel Messi,NaN,NaN,NaN,NaN,NaN,Lionel Andrés Messi Cuccitini,"Jun 24, 1987",34,left,Relatives,Paris Saint-Germain,adidas,NaN,NaN,NaN,NaN,Option for a further year,NaN,Argentina,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
32695,20/21,Ankle Injury,"Dec 23, 2020","Jan 1, 2021",9 days,1,28003,Lionel Messi,Rosario,Argentina Spain,"1,69 m",attack - Right Winger,"Jun 30, 2023",NaN,158/80,First Tier,"Aug 10, 2021","Jun 30, 2023",Lionel Messi,NaN,NaN,NaN,NaN,NaN,Lionel Andrés Messi Cuccitini,"Jun 24, 1987",34,left,Relatives,Paris Saint-Germain,adidas,NaN,NaN,NaN,NaN,Option for a further year,NaN,Argentina,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
32696,19/20,Adductor problems,"Sep 25, 2019","Oct 1, 2019",6 days,1,28003,Lionel Messi,Rosario,Argentina Spain,"1,69 m",attack - Right Winger,"Jun 30, 2023",NaN,158/80,First Tier,"Aug 10, 2021","Jun 30, 2023",Lionel Messi,NaN,NaN,NaN,NaN,NaN,Lionel Andrés Messi Cuccitini,"Jun 24, 1987",34,left,Relatives,Paris Saint-Germain,adidas,NaN,NaN,NaN,NaN,Option for a further year,NaN,Argentina,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
32697,19/20,Foot Injury,"Aug 5, 2019","Sep 16, 2019",42 days,4,28003,Lionel Messi,Rosario,Argentina Spain,"1,69 m",attack - Right Winger,"Jun 30, 2023",NaN,158/80,First Tier,"Aug 10, 2021","Jun 30, 2023",Lionel Messi,NaN,NaN,NaN,NaN,NaN,Lionel Andrés Messi Cuccitini,"Jun 24, 1987",34,left,Relatives,Paris Saint-Germain,adidas,NaN,NaN,NaN,NaN,Option for a further year,NaN,Argentina,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
32698,18/19,Pubitis,"Mar 23, 2019","Mar 29, 2019",6 days,-,28003,Lionel Messi,Rosario,Argentina Spain,"1,69 m",attack - Right Winger,"Jun 30, 2023",NaN,158/80,First Tier,"Aug 10, 2021","Jun 30, 2023",Lionel Messi,NaN,NaN,NaN,NaN,NaN,Lionel Andrés Messi Cuccitini,"Jun 24, 1987",34,left,Relatives,Paris Saint-Germain,adidas,NaN,NaN,NaN,NaN,Option for a further year,NaN,Argentina,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
32699,18/19,Fore